In [7]:
import numpy as np
import numpy.linalg as la
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy.sparse as sp
import pickle as pkl
import os
import math
import time
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [8]:
def normalized_adj(adj):
    """
    Row-normalize matrix:  A_norm = D^{-1/2} * A * D^{-1/2}.
    """
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    norm_adj = adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()
    return norm_adj.astype(np.float32)

def sparse_to_tensor(sparse_mx):
    """
    Convert a scipy sparse matrix (COO) to a TF2 SparseTensor.
    """
    sparse_mx = sparse_mx.tocoo()
    indices = np.column_stack((sparse_mx.row, sparse_mx.col))
    return tf.sparse.SparseTensor(
        indices=indices,
        values=sparse_mx.data,
        dense_shape=sparse_mx.shape
    )

def calculate_laplacian(adj):
    """
    Computes the normalized adjacency + identity, then returns a tf.sparse.SparseTensor.
    """
    # Add self-loops
    adj_normalized = normalized_adj(adj + sp.eye(adj.shape[0]))
    # Convert to SparseTensor
    return sparse_to_tensor(sp.csr_matrix(adj_normalized))

def weight_variable_glorot(input_dim, output_dim):
    """
    Used in the old GCN code. For TF2, you can simply use keras initializers directly.
    """
    init_range = np.sqrt(6.0 / (input_dim + output_dim))
    initializer = tf.random_uniform_initializer(minval=-init_range, maxval=init_range)
    return tf.Variable(initializer(shape=(input_dim, output_dim)), dtype=tf.float32)

In [ ]:
def load_sz_data():
    """
    Loads the Shenzhen dataset (adjacency and speed).
    Expects 'sz_adj.csv' and 'sz_speed.csv' in the current folder.
    """
    sz_adj = pd.read_csv('data/sz_adj.csv', header=None)
    adj = np.asmatrix(sz_adj)
    sz_tf = pd.read_csv('data/sz_speed.csv')
    return sz_tf, adj

def load_los_data():
    """
    Loads the Los Loop dataset (adjacency and speed).
    Expects 'los_adj.csv' and 'los_speed.csv' in the current folder.
    """
    los_adj = pd.read_csv('data/los_adj.csv', header=None)
    adj = np.asmatrix(los_adj)
    los_tf = pd.read_csv('data/los_speed.csv')
    return los_tf, adj

def preprocess_data(data, time_len, train_rate, seq_len, pre_len):
    """
    Generate trainX, trainY, testX, testY from the time series data.
    data: shape (time_len, num_nodes)
    """
    train_size = int(time_len * train_rate)
    train_data = data[0:train_size]
    test_data = data[train_size:time_len]
    trainX, trainY = [], []
    testX, testY = [], []
    # Build training samples
    for i in range(len(train_data) - seq_len - pre_len):
        a = train_data[i: i + seq_len + pre_len]
        trainX.append(a[0: seq_len])
        trainY.append(a[seq_len: seq_len + pre_len])
    # Build testing samples
    for i in range(len(test_data) - seq_len - pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0: seq_len])
        testY.append(b[seq_len: seq_len + pre_len])
    trainX = np.array(trainX, dtype=np.float32)
    trainY = np.array(trainY, dtype=np.float32)
    testX = np.array(testX, dtype=np.float32)
    testY = np.array(testY, dtype=np.float32)
    return trainX, trainY, testX, testY

In [10]:
def plot_result(test_result, test_label, path):
    """
    Visualize the first sensor's prediction vs. ground truth across all time,
    and also for a single day.
    test_result, test_label: arrays of shape (time, num_nodes) or (time,).
    """
    fig1 = plt.figure(figsize=(7, 1.5))
    a_pred = test_result[:, 0]
    a_true = test_label[:, 0]
    plt.plot(a_pred, 'r-', label='prediction')
    plt.plot(a_true, 'b-', label='true')
    plt.legend(loc='best', fontsize=10)
    plt.savefig(path + '/test_all.jpg')
    plt.show()
    # One day example (e.g., first 96 points)
    fig2 = plt.figure(figsize=(7, 1.5))
    a_pred_day = a_pred[:96]
    a_true_day = a_true[:96]
    plt.plot(a_pred_day, 'r-', label='prediction')
    plt.plot(a_true_day, 'b-', label='true')
    plt.legend(loc='best', fontsize=10)
    plt.savefig(path + '/test_oneday.jpg')
    plt.show()

def plot_error(train_rmse, train_loss, test_rmse, test_acc, test_mae, path):
    """Plots the metrics over epochs."""
    fig1 = plt.figure(figsize=(5,3))
    plt.plot(train_rmse, 'r-', label='train_rmse')
    plt.plot(test_rmse, 'b-', label='test_rmse')
    plt.legend(loc='best', fontsize=10)
    plt.savefig(path + '/rmse.jpg')
    plt.show()
    fig2 = plt.figure(figsize=(5,3))
    plt.plot(train_loss, 'b-', label='train_loss')
    plt.legend(loc='best', fontsize=10)
    plt.savefig(path + '/train_loss.jpg')
    plt.show()
    fig3 = plt.figure(figsize=(5,3))
    plt.plot(train_rmse, 'b-', label='train_rmse')
    plt.legend(loc='best', fontsize=10)
    plt.savefig(path + '/train_rmse.jpg')
    plt.show()
    fig4 = plt.figure(figsize=(5,3))
    plt.plot(test_acc, 'b-', label='test_acc')
    plt.legend(loc='best', fontsize=10)
    plt.savefig(path + '/test_acc.jpg')
    plt.show()
    fig5 = plt.figure(figsize=(5,3))
    plt.plot(test_rmse, 'b-', label='test_rmse')
    plt.legend(loc='best', fontsize=10)
    plt.savefig(path + '/test_rmse.jpg')
    plt.show()
    fig6 = plt.figure(figsize=(5,3))
    plt.plot(test_mae, 'b-', label='test_mae')
    plt.legend(loc='best', fontsize=10)
    plt.savefig(path + '/test_mae.jpg')
    plt.show()

In [11]:
class TGCNCell(tf.keras.layers.Layer):
    """
    Temporal Graph Convolutional Cell:
    GRU-like gating + graph convolution on [batch, num_nodes, features].
    """

    def __init__(self, num_units, adj, num_nodes, activation=tf.nn.tanh, **kwargs):
        super().__init__(**kwargs)
        self.num_units = num_units
        self.num_nodes = num_nodes
        self.activation = activation
        # Precompute sparse Laplacian
        self.adj = calculate_laplacian(adj)  # tf.sparse.SparseTensor

    @property
    def state_size(self):
        # Flattened hidden state: (batch, num_nodes * num_units)
        return self.num_nodes * self.num_units

    @property
    def output_size(self):
        # We'll output the same shape as the state per time step
        return self.num_nodes * self.num_units

    def build(self, input_shape):
        """
        input_shape: (batch, num_nodes) per time step.
        We'll create parameters for GRU gates and candidate.
        """
        # Gate parameters (r, u)
        self.w_gates = self.add_weight(
            shape=(1 + self.num_units, 2 * self.num_units),
            initializer='glorot_uniform',
            name='w_gates'
        )
        self.b_gates = self.add_weight(
            shape=(2 * self.num_units,),
            initializer='zeros',
            name='b_gates'
        )
        # Candidate parameters
        self.w_cand = self.add_weight(
            shape=(1 + self.num_units, self.num_units),
            initializer='glorot_uniform',
            name='w_cand'
        )
        self.b_cand = self.add_weight(
            shape=(self.num_units,),
            initializer='zeros',
            name='b_cand'
        )
        self.built = True

    def call(self, inputs, states, training=None):
        """
        inputs: (batch, num_nodes)
        states: list with one element = previous_state => (batch, num_nodes * num_units)
        returns: output, new_state
        """
        prev_state = states[0]  # shape: (batch, num_nodes * num_units)
        batch_size = tf.shape(inputs)[0]
        # Expand input => (batch, num_nodes, 1)
        x_in = tf.expand_dims(inputs, axis=-1)
        # Reshape state => (batch, num_nodes, num_units)
        h_prev = tf.reshape(prev_state, [batch_size, self.num_nodes, self.num_units])
        # ---- GATES ----
        # Concat => shape (batch, num_nodes, 1 + num_units)
        gates_input = tf.concat([x_in, h_prev], axis=2)
        gates = self._gconv(gates_input, self.w_gates, self.b_gates)
        gates = tf.sigmoid(gates)  # (batch, num_nodes, 2*num_units)
        r, u = tf.split(gates, 2, axis=-1)  # each => (batch, num_nodes, num_units)
        # ---- CANDIDATE ----
        r_state = r * h_prev
        cand_input = tf.concat([x_in, r_state], axis=2)
        cand = self._gconv(cand_input, self.w_cand, self.b_cand)
        if self.activation is not None:
            cand = self.activation(cand)
        # new hidden
        new_h = u * h_prev + (1.0 - u) * cand
        # Flatten => (batch, num_nodes * num_units)
        new_h_flat = tf.reshape(new_h, [batch_size, self.num_nodes * self.num_units])
        return new_h_flat, [new_h_flat]

    def _gconv(self, x_s, w, b):
        """
        Graph convolution step: multiply by adjacency, then apply w, b.
         - x_s: (batch, num_nodes, in_features)
         - w: shape (in_features, out_features)
         - b: shape (out_features,)
        Return shape: (batch, num_nodes, out_features)
        """
        batch_size = tf.shape(x_s)[0]
        in_features = tf.shape(x_s)[2]
        # => (num_nodes, in_features, batch)
        x_t = tf.transpose(x_s, [1, 2, 0])
        # => (num_nodes, in_features*batch)
        x_t = tf.reshape(x_t, [self.num_nodes, -1])
        # adjacency multiplication
        x_adj = tf.sparse.sparse_dense_matmul(self.adj, x_t)  # => (num_nodes, in_features*batch)
        # => (num_nodes, in_features, batch)
        x_adj = tf.reshape(x_adj, [self.num_nodes, in_features, batch_size])
        # => (batch, num_nodes, in_features)
        x_adj = tf.transpose(x_adj, [2, 0, 1])
        # flatten => (batch*num_nodes, in_features)
        x_adj = tf.reshape(x_adj, [batch_size * self.num_nodes, in_features])
        out = tf.matmul(x_adj, w) + b  # => (batch*num_nodes, out_features)
        out_features = tf.shape(out)[1]
        # => (batch, num_nodes, out_features)
        out = tf.reshape(out, [batch_size, self.num_nodes, out_features])
        return out

In [13]:
################################################################################
# Hyperparameters
################################################################################
learning_rate = 0.001
training_epoch = 10
gru_units = 64
seq_len = 12
pre_len = 3
train_rate = 0.8
batch_size = 32
dataset = 'los'  # or 'sz'
model_name = 'tgcn'
lambda_loss = 0.0015  # L2 factor
################################################################################
# Load Data
################################################################################
if dataset == 'sz':
    data, adj = load_sz_data()
elif dataset == 'los':
    data, adj = load_los_data()
else:
    raise ValueError("Unknown dataset")
data = np.array(data, dtype=np.float32)
time_len = data.shape[0]
num_nodes = data.shape[1]
max_value = data.max()
data = data / max_value
trainX, trainY, testX, testY = preprocess_data(data, time_len, train_rate, seq_len, pre_len)
################################################################################
# TGCN Keras Model: RNN with TGCNCell
################################################################################
class TGCNModel(tf.keras.Model):
    """
    Wrap an RNN layer using our TGCNCell, plus a final Dense to predict pre_len steps.
    """
    def __init__(self, num_units, adj, num_nodes, pre_len):
        super().__init__()
        self.num_units = num_units
        self.num_nodes = num_nodes
        self.pre_len = pre_len
        # Our TGCNCell
        self.cell = TGCNCell(num_units, adj, num_nodes)
        # RNN: unroll over seq_len
        self.rnn = tf.keras.layers.RNN(self.cell, return_sequences=True)
        # Final Dense to map from (num_nodes*units) -> (num_nodes*pre_len)
        self.out_dense = tf.keras.layers.Dense(self.num_nodes * self.pre_len)
    def call(self, inputs, training=None):
        """
        inputs shape: (batch, seq_len, num_nodes)
        Return shape: (batch*pre_len, num_nodes)
        """
        # 1) RNN => (batch, seq_len, num_nodes*units)
        rnn_outputs = self.rnn(inputs, training=training)
        # 2) Take final time step => (batch, num_nodes*units)
        last_output = rnn_outputs[:, -1, :]
        # 3) Dense => (batch, num_nodes*pre_len)
        out = self.out_dense(last_output)
        # 4) Reshape => (batch, pre_len, num_nodes)
        out = tf.reshape(out, [-1, self.pre_len, self.num_nodes])
        # => (batch, num_nodes, pre_len)
        out = tf.transpose(out, [0, 2, 1])
        # => (batch*pre_len, num_nodes)
        out = tf.reshape(out, [-1, self.num_nodes])
        return out
model = TGCNModel(gru_units, adj, num_nodes, pre_len)
################################################################################
# Prepare tf.data Datasets
################################################################################
train_size = trainX.shape[0]
test_size = testX.shape[0]
def gen_train():
    for i in range(train_size):
        yield trainX[i], trainY[i]
def gen_test():
    for i in range(test_size):
        yield testX[i], testY[i]
train_dataset = tf.data.Dataset.from_generator(
    gen_train,
    output_types=(tf.float32, tf.float32),
    output_shapes=((seq_len, num_nodes), (pre_len, num_nodes))
).batch(batch_size)
test_dataset = tf.data.Dataset.from_generator(
    gen_test,
    output_types=(tf.float32, tf.float32),
    output_shapes=((seq_len, num_nodes), (pre_len, num_nodes))
).batch(test_size)  # one batch for entire test, or break it up as you like
################################################################################
# Loss & Optimizer
################################################################################
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
def loss_fn(pred, labels, model_vars):
    # pred, labels => (batch*pre_len, num_nodes)
    mse_loss = tf.reduce_mean(tf.square(pred - labels))

    # L2 regularization
    l2_reg = 0.0
    for v in model_vars:
        l2_reg += tf.nn.l2_loss(v)
    return mse_loss + lambda_loss * l2_reg
def evaluation(a, b):
    # a, b: np arrays
    rmse = math.sqrt(mean_squared_error(a, b))
    mae = mean_absolute_error(a, b)
    F_norm = la.norm(a - b, 'fro') / la.norm(a, 'fro')
    r2 = 1 - ((a - b)**2).sum() / ((a - a.mean())**2).sum()
    var = 1 - (np.var(a - b) / np.var(a))
    return rmse, mae, 1 - F_norm, r2, var


In [16]:
import tensorflow as tf

# تعریف شکل ورودی
input_shape = (seq_len, num_nodes)  # (طول دنباله زمانی، تعداد گره‌ها)

# تعریف لایه Input
inputs = tf.keras.Input(shape=input_shape)

# ایجاد مدل TGCN
model = TGCNModel(gru_units, adj, num_nodes, pre_len)

# اتصال ورودی به مدل
outputs = model(inputs)
model.summary()
# ایجاد مدل Keras با ورودی و خروجی
tgcn_model = tf.keras.Model(inputs=inputs, outputs=outputs)

# نمایش خلاصه مدل
tgcn_model.summary()

Model: "tgcn_model_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ tgcn_cell_3 (TGCNCell)          │ ?                      │        12,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rnn_3 (RNN)                     │ (None, 12, 13248)      │        12,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 621)            │     8,227,629 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,240,301 (31.43 MB)

 Trainable params: 8,240,301 (31.43 MB)

 Non-trainable params: 0 (0.00 B)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 12, 207)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tgcn_model_3 (TGCNModel)        │ (None, 207)            │     8,240,301 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,240,301 (31.43 MB)

 Trainable params: 8,240,301 (31.43 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
################################################################################
# Training Loop
################################################################################
train_rmse_list, train_loss_list = [], []
test_rmse_list, test_mae_list, test_acc_list, test_loss_list = [], [], [], []
t_start = time.time()
for epoch in range(training_epoch):
    # ---------- Train ----------
    epoch_loss = 0.0
    epoch_rmse = 0.0
    count_batch = 0

    for batch_x, batch_y in train_dataset:
        # batch_x => (batch, seq_len, num_nodes)
        # batch_y => (batch, pre_len, num_nodes)
        with tf.GradientTape() as tape:
            y_pred = model(batch_x, training=True)  # => (batch*pre_len, num_nodes)
            y_true = tf.reshape(batch_y, [-1, num_nodes])
            l = loss_fn(y_pred, y_true, model.trainable_variables)

        grads = tape.gradient(l, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        epoch_loss += l.numpy()
        batch_rmse_val = tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true))) * max_value
        epoch_rmse += batch_rmse_val.numpy()
        count_batch += 1

    epoch_loss /= count_batch
    epoch_rmse /= count_batch
    train_loss_list.append(epoch_loss)
    train_rmse_list.append(epoch_rmse)

    # ---------- Test ----------
    for test_x, test_y in test_dataset:
        y_pred_test = model(test_x, training=False)
        y_true_test = tf.reshape(test_y, [-1, num_nodes])
        loss_test = loss_fn(y_pred_test, y_true_test, model.trainable_variables).numpy()

        pred_np = (y_pred_test.numpy()) * max_value
        true_np = (y_true_test.numpy()) * max_value
        rmse, mae, acc, r2, var = evaluation(true_np, pred_np)

        test_loss_list.append(loss_test)
        test_rmse_list.append(rmse)
        test_mae_list.append(mae)
        test_acc_list.append(acc)
        break  # end test loop

    print(f"Epoch {epoch}, Train Loss={epoch_loss:.4f}, Train RMSE={epoch_rmse:.2f}, "
          f"Test Loss={loss_test:.4f}, Test RMSE={rmse:.2f}, Test Acc={acc:.4f}")
t_end = time.time()
print("Total Training Time: %.2f s" % (t_end - t_start))
################################################################################
# Best epoch
################################################################################
best_idx = int(np.argmin(test_rmse_list))
print("Best epoch:", best_idx,
      "Test RMSE:", test_rmse_list[best_idx],
      "Test MAE:", test_mae_list[best_idx],
      "Test Acc:", test_acc_list[best_idx])
################################################################################
# Save model
################################################################################
os.makedirs("out", exist_ok=True)
model.save_weights("out/tgcn_model_weights.weights.h5")
print("Model weights saved.")

# Example retrieving final predictions
for test_x, test_y in test_dataset:
    final_preds = model(test_x, training=False).numpy() * max_value
    final_truth = test_y.numpy().reshape([-1, num_nodes]) * max_value
    break
np.savetxt("out/test_pred.csv", final_preds, delimiter=",")
np.savetxt("out/test_true.csv", final_truth, delimiter=",")
print("Saved final predictions to CSV.")